In [1]:
#| include: false
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
cd(@__DIR__)

  Activating project at `~/gitrepos/kdheepak.github.io/blog/compile-time-vs-run-time-in-julia`


In [48]:
struct Square
    side::Float64
end
area(s::Square) = s.side * s.side
    
struct Rectangle
    width::Float64
    height::Float64
end
area(r::Rectangle) = r.width * r.height
    
struct Triangle
    base::Float64
    height::Float64
end
area(t::Triangle) = 1.0/2.0 * t.base * t.height

struct Circle
    radius::Float64
end
area(c::Circle) = π * c.radius^2

area (generic function with 4 methods)

In [49]:
function shape_builder(choice::Integer)
    if choice == 1
        Square(rand())
    elseif choice == 2
        Rectangle(rand(), rand())
    elseif choice == 3
        Triangle(rand(), rand())
    elseif choice == 4
        Circle(rand())
    end
end

shape_builder (generic function with 1 method)

In [50]:
using Random
using BenchmarkTools

Random.seed!(42)
count = 1_000_000
shapes = [shape_builder(rand((1,2,3,4))) for _ in 1:count];

In [75]:
#| echo: false
using Format
using Markdown
l = cfmt("%'d", length(shapes))
Markdown.md"The total number of shapes we have is $l."

The total number of shapes we have is 1,000,000


In [51]:
function main1(shapes)
    sum(area.(shapes))
end

main1 (generic function with 1 method)

In [52]:
@time main1(shapes);

  0.084440 seconds (2.01 M allocations: 39.038 MiB, 13.30% gc time, 25.07% compilation time)


In [53]:
@benchmark main1(shapes)

BenchmarkTools.Trial: 117 samples with 1 evaluation.
 Range (min … max):  41.874 ms …  44.243 ms  ┊ GC (min … max): 0.00% … 2.69%
 Time  (median):     42.828 ms               ┊ GC (median):    1.10%
 Time  (mean ± σ):   42.832 ms ± 489.585 μs  ┊ GC (mean ± σ):  1.04% ± 0.80%

                    ▇  ▂  ▁█ ▁                                  
  ▃▃▁▃▃▆█▁▆▆▁█▁▅▅▅▁██▆▅█▅▆████▅▃██▆▅▃▅▅█▅▁▁▅▁▁▁▁▃▁▁▁▃▅▁▁▁▃▃▁▁▃ ▃
  41.9 ms         Histogram: frequency by time         44.2 ms <

 Memory estimate: 38.14 MiB, allocs estimate: 1999492.

In [54]:
filter_shapes(T, shapes) = Any[shape for shape in filter(x -> isa(x, T), shapes)]
arr1 = filter_shapes(Square, shapes)
arr2 = filter_shapes(Rectangle, shapes)
arr3 = filter_shapes(Triangle, shapes)
arr4 = filter_shapes(Circle, shapes)
function main2(arr1, arr2, arr3, arr4)
    sum(area.(arr1)) + sum(area.(arr2)) + sum(area.(arr3)) + sum(area.(arr4))
end

main2 (generic function with 3 methods)

In [55]:
@time main2(arr1, arr2, arr3, arr4);

  0.039215 seconds (2.00 M allocations: 38.116 MiB, 9.11% gc time)


In [56]:
@benchmark main2(arr1, arr2, arr3, arr4)

BenchmarkTools.Trial: 146 samples with 1 evaluation.
 Range (min … max):  33.388 ms …  36.605 ms  ┊ GC (min … max): 0.00% … 4.67%
 Time  (median):     34.319 ms               ┊ GC (median):    1.16%
 Time  (mean ± σ):   34.444 ms ± 644.855 μs  ┊ GC (mean ± σ):  1.36% ± 1.14%

          ▁   ▂  ▄▇ ▄▂ ▄▂█                                      
  ▅▅▆▁▃▅▃▆█▅███▅███▁███████▃▆▆▅▃▃▃▃▃▆▃▅▆▅▆▅▃▁▁▅▁▁▁▅▁▁▃▁▃▃▃▁▃▃▆ ▃
  33.4 ms         Histogram: frequency by time         36.1 ms <

 Memory estimate: 38.12 MiB, allocs estimate: 1997965.

In [60]:
filter_shapes(T, shapes) = T[shape for shape in filter(x -> isa(x, T), shapes)]
arr1 = filter_shapes(Square, shapes)
arr2 = filter_shapes(Rectangle, shapes)
arr3 = filter_shapes(Triangle, shapes)
arr4 = filter_shapes(Circle, shapes)
nothing # hide

In [58]:
@time main2(arr1, arr2, arr3, arr4);

  0.037636 seconds (127.94 k allocations: 16.146 MiB, 97.72% compilation time)


In [59]:
@benchmark main2(arr1, arr2, arr3, arr4)

BenchmarkTools.Trial: 4293 samples with 1 evaluation.
 Range (min … max):  482.083 μs …   5.092 ms  ┊ GC (min … max):  0.00% … 79.04%
 Time  (median):     936.542 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.159 ms ± 625.098 μs  ┊ GC (mean ± σ):  20.15% ± 22.12%

     ▃▄▄██▇▇▆▅▅▄▁         ▁▁▁▁▁ ▁▁▁▂▁  ▁                        ▂
  ▇▅█████████████▆▁▃▁▁▃▁▄▇████████████████▇▇▇▆▇▇█▇▆▆█▄▇▆▆▇▅▆▅▆▅ █
  482 μs        Histogram: log(frequency) by time       3.73 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 9.